## Finetune Flair

In [1]:
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd

import torch

from flair.data import Corpus, Sentence
from flair.datasets import ClassificationCorpus
from flair.embeddings import (
    WordEmbeddings,
    FlairEmbeddings,
    DocumentRNNEmbeddings,
    TransformerWordEmbeddings,
    StackedEmbeddings,
)
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric
from flair.visual.training_curves import Plotter

import tqdm

import flair
flair.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {flair.device}")

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

c:\Users\Ning\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.5.1+cu124
CUDA Available: True
CUDA Version: 12.4


In [3]:
# Ensure the output directory exists
flair_data_folder = "data/flair_data"
os.makedirs(flair_data_folder, exist_ok=True)

# Define the folder where the data is located
corpus_folder = Path(flair_data_folder)

# Create the corpus
corpus = ClassificationCorpus(
    corpus_folder,
    train_file='train.txt',
    dev_file='dev.txt',
    test_file='test.txt',
    label_type='sentiment'
)

# Print statistics
print(f"Number of training sentences: {len(corpus.train)}")
print(f"Number of validation sentences: {len(corpus.dev)}")
print(f"Number of test sentences: {len(corpus.test)}")

2024-12-03 18:21:18,430 Reading data from data\flair_data
2024-12-03 18:21:18,430 Train: data\flair_data\train.txt
2024-12-03 18:21:18,430 Dev: data\flair_data\dev.txt
2024-12-03 18:21:18,430 Test: data\flair_data\test.txt
2024-12-03 18:21:18,956 Initialized corpus data\flair_data (label type name is 'sentiment')
Number of training sentences: 22500
Number of validation sentences: 2500
Number of test sentences: 25000


In [4]:
label_dict = corpus.make_label_dictionary(label_type='sentiment')
print(label_dict)

2024-12-03 18:21:18,960 Computing label dictionary. Progress:


0it [00:00, ?it/s]
22500it [00:42, 527.47it/s]

2024-12-03 18:22:01,633 Dictionary created for label 'sentiment' with 2 values: POS (seen 11298 times), NEG (seen 11202 times)
Dictionary with 2 tags: POS, NEG


In [5]:
# Step 4: Set Up Stacked Embeddings
flair_forward_embedding = FlairEmbeddings('news-forward')
flair_backward_embedding = FlairEmbeddings('news-backward')
transformer_word_embeddings = TransformerWordEmbeddings('distilbert-base-uncased')

# List of embeddings
embeddings = [
    flair_forward_embedding,
    flair_backward_embedding,
    transformer_word_embeddings,
]

# Create document embeddings from word embeddings
document_embeddings = DocumentRNNEmbeddings(
    embeddings=embeddings,
    hidden_size=256,
    reproject_words=True,
    reproject_words_dimension=256,
)

In [6]:
classifier = TextClassifier(
    document_embeddings,
    label_dictionary=label_dict,
    label_type='sentiment'
).to(flair.device)

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import tqdm
from collections import Counter

def evaluate_model(classifier, test_dataset):
    """
    Evaluate a Flair classifier on a given test dataset, with verification and debugging steps.

    Args:
        classifier (TextClassifier): The trained Flair classifier.
        test_dataset (Dataset): The test dataset.

    Returns:
        dict: Dictionary containing evaluation metrics.
    """
    true_labels = []
    predicted_labels = []

    # Label mapping
    label_mapping = {"NEG": 0, "POS": 1}

    # Iterate over the test dataset with tqdm progress bar
    for sentence in tqdm.tqdm(test_dataset, desc="Evaluating", leave=True):
        sentence.to(flair.device)

        # Get true label
        true_label = sentence.get_label("sentiment").value
        true_labels.append(true_label)

        # Get predicted label
        classifier.predict(sentence)
        predicted_label = sentence.labels[0].value
        predicted_labels.append(predicted_label)

    # Verify label consistency
    print("True Labels Sample:", true_labels[:5])
    print("Predicted Labels Sample:", predicted_labels[:5])
    print("True Label Distribution:", Counter(true_labels))
    print("Predicted Label Distribution:", Counter(predicted_labels))

    # Map labels to numeric values for sklearn
    try:
        true_labels_mapped = [label_mapping[label] for label in true_labels]
        predicted_labels_mapped = [label_mapping[label] for label in predicted_labels]
    except KeyError as e:
        print(f"Label mapping error: {e}. Ensure all labels are in {label_mapping}.")
        return {}

    # Verify mapped labels
    print("Mapped True Labels Sample:", true_labels_mapped[:5])
    print("Mapped Predicted Labels Sample:", predicted_labels_mapped[:5])

    # Calculate metrics
    accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
    precision = precision_score(true_labels_mapped, predicted_labels_mapped, pos_label=1, zero_division=0)
    recall = recall_score(true_labels_mapped, predicted_labels_mapped, pos_label=1, zero_division=0)
    f1 = f1_score(true_labels_mapped, predicted_labels_mapped, pos_label=1, zero_division=0)

    # Full classification report
    classification_rep = classification_report(
        true_labels_mapped,
        predicted_labels_mapped,
        target_names=["NEG", "POS"]  # Match target names with label_mapping
    )

    # Print metrics
    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_rep)

    # Return metrics as a dictionary
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "classification_report": classification_rep,
    }

In [8]:
from torch.optim import AdamW
import logging


# Test on small dataset
small_data_folder = "data/flair_data"
small_corpus = ClassificationCorpus(small_data_folder, 
                                    train_file="train_small.txt",  
                                    dev_file='dev_small.txt', 
                                    test_file='test_small.txt',
                                    label_type="sentiment")


# Set the logging level to INFO
logging.basicConfig(level=logging.INFO)

# Initialize the trainer
trainer = ModelTrainer(classifier, corpus)

#Fine-tune the model
trainer.fine_tune(
    base_path='flair_model',             # Directory to save the model and logs
    learning_rate=5e-5,                  # Learning rate for fine-tuning
    mini_batch_size=16,                   # Smaller batch size for transformers
    max_epochs=10,                        # Number of epochs
    embeddings_storage_mode='gpu',     
    optimizer=AdamW,                     # Optimizer suited for transformers
    save_final_model=True,               # Save the final model
    save_model_each_k_epochs=1,          # Save model checkpoint every epoch
    create_file_logs=True,               # Save logs to a file
    create_loss_file=True,               # Save loss values to a file
    use_final_model_for_eval=False
)



2024-12-03 18:22:03,826 Reading data from data\flair_data
2024-12-03 18:22:03,827 Train: data\flair_data\train_small.txt
2024-12-03 18:22:03,827 Dev: data\flair_data\dev_small.txt
2024-12-03 18:22:03,827 Test: data\flair_data\test_small.txt
2024-12-03 18:22:03,831 Initialized corpus data/flair_data (label type name is 'sentiment')
2024-12-03 18:22:03,833 ----------------------------------------------------------------------------------------------------
2024-12-03 18:22:03,834 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )


c:\Users\Ning\AppData\Local\Programs\Python\Python312\Lib\site-packages\flair\trainers\trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-12-03 18:40:44,374 epoch 1 - iter 140/1407 - loss 0.70971192 - time (sec): 1120.53 - samples/sec: 2.00 - lr: 0.000005 - momentum: 0.000000
2024-12-03 18:59:58,059 epoch 1 - iter 280/1407 - loss 0.67125986 - time (sec): 2274.22 - samples/sec: 1.97 - lr: 0.000010 - momentum: 0.000000
2024-12-03 19:24:28,877 epoch 1 - iter 420/1407 - loss 0.56417752 - time (sec): 3745.03 - samples/sec: 1.79 - lr: 0.000015 - momentum: 0.000000
2024-12-03 19:47:53,884 epoch 1 - iter 560/1407 - loss 0.50285825 - time (sec): 5150.04 - samples/sec: 1.74 - lr: 0.000020 - momentum: 0.000000
2024-12-03 20:10:01,058 epoch 1 - iter 700/1407 - loss 0.46016501 - time (sec): 6477.22 - samples/sec: 1.73 - lr: 0.000025 - momentum: 0.000000
2024-12-03 20:31:58,632 epoch 1 - iter 840/1407 - loss 0.43516031 - time (sec): 7794.79 - samples/sec: 1.72 - lr: 0.000030 - momentum: 0.000000
2024-12-03 20:50:19,360 epoch 1 - iter 980/1407 - loss 0.41024027 - time (sec): 8895.52 - samples/sec: 1.76 - lr: 0.000035 - momentum: 0

100%|██████████| 157/157 [32:51<00:00, 12.56s/it]

2024-12-03 22:38:48,419 DEV : loss 0.33744463324546814 - f1-score (micro avg)  0.9004


...

2024-12-06 13:01:59,773 DEV : loss 0.7704070210456848 - f1-score (micro avg)  0.9284


2024-12-06 13:02:05,071 ----------------------------------------------------------------------------------------------------
2024-12-06 13:02:05,072 Loading model from best epoch ...


100%|██████████| 1563/1563 [4:18:48<00:00,  9.94s/it]  


2024-12-06 17:20:55,301 
Results:
- F-score (micro) 0.929
- F-score (macro) 0.929
- Accuracy 0.929

By class:
              precision    recall  f1-score   support

         POS     0.9208    0.9387    0.9297     12500
         NEG     0.9375    0.9193    0.9283     12500

    accuracy                         0.9290     25000
   macro avg     0.9292    0.9290    0.9290     25000
weighted avg     0.9292    0.9290    0.9290     25000

2024-12-06 17:20:55,302 ----------------------------------------------------------------------------------------------------


{'test_score': 0.929}

In [9]:
# Path to the saved model
saved_model_path = "flair_model/best-model.pt"  # Update this if the path or filename is different

# Load the trained model
classifier = TextClassifier.load(saved_model_path).to(flair.device)

# Evaluate the model
results = evaluate_model(classifier, corpus.test)


Evaluating: 100%|██████████| 25000/25000 [3:56:31<00:00,  1.76it/s]  

True Labels Sample: ['NEG', 'NEG', 'NEG', 'NEG', 'NEG']
Predicted Labels Sample: ['POS', 'NEG', 'NEG', 'NEG', 'NEG']
True Label Distribution: Counter({'NEG': 12500, 'POS': 12500})
Predicted Label Distribution: Counter({'POS': 12743, 'NEG': 12257})
Mapped True Labels Sample: [0, 0, 0, 0, 0]
Mapped Predicted Labels Sample: [1, 0, 0, 0, 0]

Accuracy: 0.9290
Precision: 0.9208
Recall: 0.9387
F1 Score: 0.9297

Classification Report:
              precision    recall  f1-score   support

         NEG       0.94      0.92      0.93     12500
         POS       0.92      0.94      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000

